In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import astropy
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join
import desispec

# If not using the desiconda version of prospect: EDIT THIS to your path
# sys.path.insert(0,"/global/homes/b/bid13/DESI/DESI-II/prospect/py/") 
from prospect import viewer, utilities

In [ ]:
data_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/pilot_obs/COSMOS/")

In [ ]:
cat = Table.read(data_path / "merged_cat.fits")
mask = (np.isin(cat["TERTIARY_TARGET"].astype(str), ["MERIAN_MS1", "MERIAN_MS2", "MERIAN_MS3"]))
cat = cat[mask]

cat.sort("DELTACHI2")
cat.reverse()

In [ ]:
# np.unique(cat['TERTIARY_TARGET'])

In [ ]:
plt.hist(cat['Z'], bins=100, range=(0,2))
plt.yscale('log')

In [ ]:
all_spectra = desispec.io.read_spectra(data_path / "spectra.fits")


all_redrock = Table.read(data_path / "zbest.fits")
all_redrock[all_redrock["SUBTYPE"].mask]["SUBTYPE"] = b'NA'
all_redrock["SUBTYPE"].mask =False

for c in all_redrock.columns:
    # print(c)
    if isinstance(all_redrock[c], astropy.table.column.MaskedColumn):
        all_redrock[c] = all_redrock[c].value.data
all_redrock.add_index("TARGETID")

rr_details = Table.read(data_path / "redrock_details.fits")
for c in rr_details.columns:
    if c == "SPECTYPE":
        rr_details[c] = rr_details[c].value.astype(str)
    if c =="SUBTYPE":
        rr_details[c] = rr_details[c].value.data.astype(str)
rr_details.add_index("TARGETID")


### Generate all pages

In [ ]:
def create_vi_page(sel_mask, file_name ):
    
    
    all_targets = cat["TARGETID"][sel_mask].value
    for i in range(0,len(all_targets.data), 50):
        sel_targets = all_targets[i:i+50]
        sel_spectra = all_spectra.select(targets = sel_targets)
        sel_targets = sel_spectra.fibermap["TARGETID"].value
        sel_zcat = all_redrock.loc[sel_targets]
        sel_rr_det = rr_details.loc[sel_targets]

        viewer.plotspectra(sel_spectra, 
                           zcatalog=sel_zcat,
                           redrock_cat=sel_rr_det,
                           notebook=False, title=f'{file_name}_{(i//50)+1}', 
                           html_dir = f"/global/cfs/cdirs/desi/users/bid13/DESI_II/VI_pages/MERIAN/"
                           # mask_type='SV1_DESI_TARGET'
                          )

In [ ]:
mask = (cat["TERTIARY_TARGET"] == 'MERIAN_MS1')
create_vi_page(mask, "MERIAN_MS1")
mask.sum()

In [ ]:
mask = (cat["TERTIARY_TARGET"] == 'MERIAN_MS2')
create_vi_page(mask,"MERIAN_MS2")
mask.sum()

In [ ]:
mask = (cat["TERTIARY_TARGET"] == 'MERIAN_MS3')
create_vi_page(mask,"MERIAN_MS3")
mask.sum()